In [ ]:
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Dependencies
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

import os, sys, base64
from scipy.io import wavfile
import pathlib
from tqdm import tqdm
ROOT = pathlib.Path(__file__).parent
sys.path.append(str(ROOT / '..'))
sys.path.append(str(object=ROOT ))
from common.errors import ConfigurationError
from common import graphing

# Load Edge impulse Processing Block SpeechPy fork
MODULE_PATH = os.path.join(os.path.dirname(os.path.realpath(__file__)), 'third_party', 'speechpy')
MODULE_NAME = 'speechpy'
import importlib
spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
speechpy = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = speechpy
spec.loader.exec_module(speechpy)



class Process_Audio_Data():
    """
    Class used to preprocess audio data into MFE and MFCC spectrograms.
    """
    def __init__(self,
                 data_path,
                 implementation_version, draw_graphs, axes, 
                 frame_length, frame_stride, num_filters, fft_length, noise_floor_db,
                 num_cepstral, win_size,
                 low_frequency, high_frequency,
                 pre_cof, pre_shift):
        
        self.data_path = data_path
        self.implementation_version = implementation_version
        self.draw_graphs = draw_graphs
        self.axes = axes
        self.frame_length = frame_length
        self.frame_stride = frame_stride
        self.num_filters = num_filters
        self.fft_length = fft_length
        self.num_cepstral = num_cepstral
        self.win_size = win_size
        self.low_frequency = low_frequency 
        self.high_frequency = high_frequency
        self.pre_cof = pre_cof
        self.pre_shift = pre_shift
        self.noise_floor_db = noise_floor_db
        self.zero_padding = zero_padding

        self.mfcc_features = []
        self.mfe_features = []

        self.mfcc_graphs = {}
        self.mfe_graphs = {}

        self.data_info = pd.DataFrame(columns=['unique_id','label','noise_type','gender','device_type'])

        self.width = None
        self.height = None

    
    def generate_mfcc_features(self, raw_data, fs):
        """
        Function generates MFCC features, code taken from Edge Impulse Processing Block repo. 
        
        From: https://github.com/edgeimpulse/processing-blocks/tree/master
        """
        features = []
        graphs = []

        for ax in range(0, len(self.axes)):
            signal = raw_data[:,ax]

            numframes, _, __ = speechpy.processing.calculate_number_of_frames(
                signal,
                implementation_version=self.implementation_version,
                sampling_frequency=fs,
                frame_length=self.frame_length,
                frame_stride=self.frame_stride,
                zero_padding=self.zero_padding)

            if (numframes < 1):
                raise ConfigurationError('Frame length is larger than your window size')

            if (numframes > 500):
                raise ConfigurationError('Number of frames is larger than 500 (' + str(numframes) + '), ' +
                    'increase your frame stride or decrease your window size.')

            # Example of pre-emphasizing.
            signal_preemphasized = speechpy.processing.preemphasis(signal, cof=self.pre_cof, shift=self.pre_shift)

            ############# Extract MFCC features #############
            use_old_mels = True if self.implementation_version <= 3 else False
            mfcc = speechpy.feature.mfcc(signal_preemphasized, sampling_frequency=fs, implementation_version=self.implementation_version,
                        frame_length=self.frame_length,
                        frame_stride=self.frame_stride, num_filters=self.num_filters, fft_length=self.fft_length,
                        num_cepstral=self.num_cepstral,
                        low_frequency=self.low_frequency, high_frequency=self.high_frequency,
                        use_old_mels=use_old_mels)
            if self.win_size > 0:
                mfcc_cmvn = speechpy.processing.cmvnw(mfcc, win_size=self.win_size, variance_normalization=True)
            else:
                mfcc_cmvn = mfcc

            flattened = mfcc_cmvn.flatten()
            features = np.concatenate((features, flattened))

            if self.draw_graphs:
                # make visualization too
                fig, ax = plt.subplots()
                fig.set_size_inches(18.5, 20.5)
                # swap to have time on long axis (x)
                # flip so we go from low to high cepstrums
                mfcc_data = np.swapaxes(np.flip(mfcc_cmvn, 1), 0 ,1)
                graphing.set_x_axis_times(self.frame_stride, self.frame_length, np.shape(mfcc_data)[1])
                height = np.shape(mfcc_data)[0]
                y_pos = range(0,height)
                y_nums = range(height,0,-1)
                plt.yticks(y_pos,y_nums)
                ax.imshow(mfcc_data, interpolation='nearest', cmap=cm.coolwarm)

                buf = io.BytesIO()
                plt.savefig(buf, format='svg', bbox_inches='tight', pad_inches=0)

                buf.seek(0)
                image = (base64.b64encode(buf.getvalue()).decode('ascii'))

                buf.close()

                graphs.append({
                    'name': 'Cepstral Coefficients',
                    'image': image,
                    'imageMimeType': 'image/svg+xml',
                    'type': 'image'
                })

        return {
            'features': features.tolist(),
            'graphs': graphs,
            'fft_used': [ self.num_filters, self.fft_length ],
            'output_config': {
                'type': 'spectrogram',
                'shape': {
                    'width': len(features) / self.num_cepstral,
                    'height': self.num_cepstral
                }
            }
        }
    
    def generate_mfe_features(self, raw_data, fs):
        """
        Function generates MFE features, code taken from Edge Impulse Processing Block repo. 
        From: https://github.com/edgeimpulse/processing-blocks/tree/master
        """
        width = 0
        height = 0

        for ax in range(0, len(self.axes)):
            signal = raw_data[:,ax]

            if self.implementation_version >= 3:
                # Rescale to [-1, 1] and add preemphasis
                signal = (signal / 2**15).astype(np.float32)
                signal = speechpy.processing.preemphasis(signal, cof=0.98, shift=1)

            # check for too many frames
            numframes, _, __ = speechpy.processing.calculate_number_of_frames(
                signal,
                implementation_version=self.implementation_version,
                sampling_frequency=fs
                frame_length=self.frame_length,
                frame_stride=self.frame_stride,
                zero_padding=self.zero_padding)

            if (numframes < 1):
                raise ConfigurationError('Frame length is larger than your window size')

            if (numframes > 500):
                raise ConfigurationError('Number of frames is larger than 500 (' + str(numframes) + '), ' +
                    'increase your frame stride or decrease your window size.')
            
            ############# Extract MFE features #############
            use_old_mels = True if self.implementation_version <= 3 else False
            mfe, _, filterbank_freqs, filterbank_matrix = speechpy.feature.mfe(signal, sampling_frequency=fs, implementation_version=self.implementation_version,
                                            frame_length=self.frame_length,
                                            frame_stride=self.frame_stride, num_filters=self.num_filters, fft_length=self.fft_length,
                                            low_frequency=self.low_frequency, high_frequency=self.high_frequency,
                                            use_old_mels=use_old_mels)

            if self.implementation_version < 3:
                mfe_2d = speechpy.processing.cmvnw(mfe, win_size=self.win_size, variance_normalization=False)

                if (np.min(mfe_2d) != 0 and np.max(mfe_2d) != 0):
                    mfe_2d = (mfe_2d - np.min(mfe_2d)) / (np.max(mfe_2d) - np.min(mfe_2d))

                mfe_2d[np.isnan(mfe_2d)] = 0

                flattened = mfe_2d.flatten()
            else:
                # Clip to avoid zero values
                mfe = np.clip(mfe, 1e-30, None)
                # Convert to dB scale
                # log_mel_spec = 10 * log10(mel_spectrograms)
                mfe = 10 * np.log10(mfe)

                # Add power offset and clip values below 0 (hard filter)
                # log_mel_spec = (log_mel_spec + self._power_offset - 32 + 32.0) / 64.0
                # log_mel_spec = tf.clip_by_value(log_mel_spec, 0, 1)
                mfe = (mfe - self.noise_floor_db) / ((-1 * self.noise_floor_db) + 12)
                mfe = np.clip(mfe, 0, 1)

                # Quantize to 8 bits and dequantize back to float32
                mfe = np.uint8(np.around(mfe * 2**8))
                # clip to 2**8
                mfe = np.clip(mfe, 0, 255)
                mfe = np.float32(mfe / 2**8)

                mfe_2d = mfe

                flattened = mfe.flatten()

            features = np.concatenate((features, flattened))

            width = np.shape(mfe)[0]
            height = np.shape(mfe)[1]

            if self.draw_graphs:
                image = graphing.create_mfe_graph(
                    fs, self.frame_length, self.frame_stride, width, height, np.swapaxes(mfe_2d, 0, 1), filterbank_freqs)

                image2 = graphing.create_graph(filterbank_matrix, "Output Row Index", "FFT Bin Index")

                self.graphs.append({
                    'name': 'Mel Energies (DSP Output)',
                    'image': image,
                    'imageMimeType': 'image/svg+xml',
                    'type': 'image'
                })

                self.graphs.append({
                    'name': 'FFT Bin Weighting',
                    'image': image2,
                    'imageMimeType': 'image/svg+xml',
                    'type': 'image'
                })

        return {
            'features': features.tolist(),
            'graphs': self.graphs,
            'fft_used': [ self.fft_length ],
            'output_config': {
                'type': 'spectrogram',
                'shape': {
                    'width': width,
                    'height': height
                }
            }
        }


    def generate_features(self):
        """
        Function that organizes passed in input data and processes information and saves generated features
        for MFE and MFCC as well as obtain labels and save generated graphs to proper variables.
        
        From: https://github.com/edgeimpulse/processing-blocks/tree/master
        """
        
        # looping over audio files
        for filename in tqdm(os.listdir(self.data_path)):
            if filename.endswith(".wav"):
                file_path = os.path.join(self.data_path, filename)

                # read in raw audio data
                sampling_freq, raw_data = wavfile.read(file_path)
                
                if (self.implementation_version > 4):
                    raise Exception('implementation_version should be between 1 and 4')

                if (self.num_filters < 2):
                    raise ConfigurationError('Filter number should be at least 2')
                if (not math.log2(self.fft_length).is_integer()):
                    raise ConfigurationError('FFT length must be a power of 2')
                if (len(self.axes) != 1):
                    raise ConfigurationError('MFCC blocks only support a single axis, ' +
                        'create one MFCC block per axis under **Create impulse**')
                
                fs = sampling_freq
                high_frequency = None if high_frequency == 0 else high_frequency

                # reshape first
                raw_data = raw_data.reshape(int(len(raw_data) / len(self.axes)), len(self.axes))
                
                # generate MFE features
                mfe_output = self.generate_mfe_features(raw_data, fs)
                self.mfe_features.append(mfe_output['features']) # appending to global list 

                # generate MFCC features
                mfcc_output = self.generate_mfcc_features(raw_data, fs)
                self.mfe_features.append(mfcc_output['features']) # appending to global list 

                # extracting file name informations (including label)
                file_name_parts = filename.split('.')  # Adjust if your labeling system is different
                label = file_name_parts[0]
                noise_type = file_name_parts[1]
                gender = file_name_parts[2]
                device_type = file_name_parts[3]
                unique_id = file_name_parts[4]
                
                row = pd.DataFrame({
                'unique_id':[unique_id], 
                'label':[label], 
                'noise_type':[noise_type], 
                'gender':[gender], 
                'device_type':[device_type]})

                self.data_info = pd.concat([self.data_info, row]) # adding new data information to dataframe  
                
                # saving graph information 
                self.mfcc_graphs[unique_id] = mfcc_output['graphs']
                self.mfe_graphs[unique_id] = mfe_output['graphs']

        self.width = mfe_output['output_config']['shape']['width']
        self.height = mfe_output['output_config']['shape']['height']